# Question and problem definition
#### Submit predicted probabilities for every possible matchup in the past 5 NCAA® tournaments (2016-2019 and 2021). Note that there was no tournament held in 2020.

# Import packages

In [69]:
import os
import re
import sklearn
import numpy as np 
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import matplotlib.pyplot as plt

from collections import Counter
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.model_selection import *
# for model
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

pd.set_option('display.max_columns', None)

# Load the data

In [25]:
DATA_PATH = '../input/mens-march-mania-2022/MDataFiles_Stage1/'

for filename in os.listdir(DATA_PATH):
    print(filename)

## Data Section 1 - The Basics

## df_teams
+ TeamID
+ Teams name
+ FirstD1Season
+ LastD1Season

In [26]:
df_teams = pd.read_csv(DATA_PATH + "MTeams.csv")
df_teams.head()

## df_seasons
+ Season : 比賽年分
+ DayZero : 賽季第0天/開始日期(DayNum)
+ RegionW / RegionX / RegionY / RegionZ : 攸關seeds中種子命名規則 


In [27]:
df_seasons = pd.read_csv(DATA_PATH + "MSeasons.csv")
df_seasons.head()

## df_seeds
+ Seed : W/X/Y/Z表示Region

In [28]:
df_seeds = pd.read_csv(DATA_PATH + "MNCAATourneySeeds.csv")
df_seeds.head()

## df_regular
* All regular game dataframe
* DayNum : 賽季開始日期為DayNum = 0 ，DayNum 132之前為例行賽
* WTeamID : 勝利球隊ID
* LTeamID : 失敗球隊ID
* WLoc : 勝利球隊是主場還客場，H = 主隊勝、N = 中立場、A = 客隊勝

In [29]:
df_regular = pd.read_csv(DATA_PATH + "MRegularSeasonCompactResults.csv")
df_regular.head()

In [30]:
df_regular['WLoc'].value_counts()

In [31]:
 df_regular['NumOT'].value_counts()

In [32]:
fig1 = go.Figure()
fig1.add_trace(go.Histogram(x = df_regular.WScore, name = 'Win'))
fig1.add_trace(go.Histogram(x = df_regular.LScore, name = 'Lose'))

# Overlay both histograms
fig1.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig1.update_traces(opacity=0.75)
fig1.show()

In [33]:
# if some team were better in OT
OTbetter = df_regular.loc[df_regular['NumOT'] > 0]
OTbetter['WTeamID'].value_counts() # it seems did have some teams display better in OT, team culture might play a role ? 

In [34]:
fig2 = px.box(OTbetter, x = OTbetter['WTeamID'].value_counts())
fig2.show()
# if win in OT more than 23 games given 1 others given 0

## Data Section 2 - Team Box Scores

## df_season_results
* WFGM - field goals made (by the winning team)
* WFGA - field goals attempted (by the winning team)
* WFGM3 - three pointers made (by the winning team)
* WFGA3 - three pointers attempted (by the winning team)
* WFTM - free throws made (by the winning team)
* WFTA - free throws attempted (by the winning team)
* WOR - offensive rebounds (pulled by the winning team)
* WDR - defensive rebounds (pulled by the winning team)
* WAst - assists (by the winning team)
* WTO - turnovers committed (by the winning team)
* WStl - steals (accomplished by the winning team)
* WBlk - blocks (accomplished by the winning team)
* WPF - personal fouls committed (by the winning team)

In [35]:
df_season_results = pd.read_csv(DATA_PATH + "MRegularSeasonDetailedResults.csv")
df_season_results['Scoredistance'] = df_season_results['WScore'] - df_season_results['LScore']

In [36]:
df_season_results.head()

## df_TourneyCompact

In [37]:
df_TourneyCompact = pd.read_csv(DATA_PATH + "MNCAATourneyCompactResults.csv")
df_TourneyCompact.head()

# Data Cleaning

## Combine dataframe sabermetrics

In [38]:
display(df_season_results.columns.values)

In [39]:
# set up empty df
Winteams = pd.DataFrame()
Loseteams = pd.DataFrame()

# set up col
col = ['Season', 'TeamID', 'Points', 'OppPoints',
       'Loc', 'NumOT', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA',
       'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'OppFGM', 'OppFGA',
       'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppOR', 'OppDR', 'OppAst', 'OppTO',
       'OppStl', 'OppBlk', 'OppPF', 'Scoredistance']

# build Winteams and Loseteams df
## Winteams
Winteams[col] = df_season_results[['Season', 'WTeamID', 'WScore', 'LScore',
       'WLoc', 'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA',
       'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA',
       'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO',
       'LStl', 'LBlk', 'LPF', 'Scoredistance']]

Winteams['Win'] = 1 
Winteams['Lose'] = 0


## Loseteams
Loseteams[col] = df_season_results[['Season', 'LTeamID', 'LScore','WScore',
       'WLoc', 'NumOT', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA',
       'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'WFGM', 'WFGA',
       'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO',
       'WStl', 'WBlk', 'WPF', 'Scoredistance']]

def change_loc(loc):
    if loc == 'H':
        return 'A'
    elif loc == 'A':
        return 'H'
    else:
        return 'N'
    
Loseteams['Loc'] = Loseteams['Loc'].apply(change_loc)

Loseteams['Win'] = 0
Loseteams['Lose'] = 1

sabermetrics = pd.concat([Winteams, Loseteams])
sabermetrics = sabermetrics.groupby(['Season', 'TeamID']).sum()
sabermetrics['NumGames'] = sabermetrics['Win'] + sabermetrics['Lose']

In [40]:
RegularSeasonInput = pd.DataFrame()

# Winratio
RegularSeasonInput['WinRatio'] = sabermetrics['Win'] / sabermetrics['NumGames']

# PointPerGame
RegularSeasonInput['PointPerGame'] = sabermetrics['Points'] / sabermetrics['NumGames']
RegularSeasonInput['PointAllowPerGame'] = sabermetrics['OppPoints'] / sabermetrics['NumGames']

# Possessions Per Game(每場比賽回合數 & 每場比賽對手回合數 -> 可看出節奏) 
RegularSeasonInput['Possessions'] = ((sabermetrics['FGA'] - sabermetrics['OR']) + sabermetrics['TO'] + .44 * sabermetrics['FTA']) / sabermetrics['NumGames']
RegularSeasonInput['OppPossessions'] = ((sabermetrics['OppFGA'] - sabermetrics['OppOR']) + sabermetrics['OppTO'] + .44 * sabermetrics['OppFTA']) / sabermetrics['NumGames']

# Points Per Possession(每回合得分 & 對手每回合得分)
RegularSeasonInput['PtsPerPoss'] = sabermetrics['Points'] / RegularSeasonInput['Possessions']
RegularSeasonInput['OppPtsPerPoss'] = sabermetrics['OppPoints'] / RegularSeasonInput['OppPossessions']

# Effective Field Goal Percentage(有效命中率 & 對手有效命中率)
RegularSeasonInput['EffectiveFGPct'] = ((sabermetrics['Points'] - sabermetrics['FTM']) / 2) / sabermetrics['FGA']
RegularSeasonInput['OppEffectiveFGPct'] = ((sabermetrics['OppPoints'] - sabermetrics['OppFTM']) / 2) / sabermetrics['OppFGA']

# Percentage of Field Goals Assisted(助攻率 & 對手助攻率)
RegularSeasonInput['AssistRate'] = sabermetrics['Ast'] / sabermetrics['FGM']
RegularSeasonInput['OppAssistRate'] = sabermetrics['OppAst'] / sabermetrics['OppFGM']

# Offensive Rebound Percentage 
RegularSeasonInput['OReboundPct'] = sabermetrics['OR'] / (sabermetrics['FGA'] - sabermetrics['FGM'])
RegularSeasonInput['OppOReboundPct'] = sabermetrics['OppOR'] / (sabermetrics['OppFGA'] - sabermetrics['OppFGM'])

# Defensive Rebound Percentage
RegularSeasonInput['DReboundPct'] = sabermetrics['DR'] / (sabermetrics['OppFGA'] - sabermetrics['OppFGM'])
RegularSeasonInput['OppDReboundPct'] =sabermetrics['OppDR'] / (sabermetrics['FGA'] - sabermetrics['FGM'])

# Assist to Turnover Ratio
RegularSeasonInput['ATORatio'] = sabermetrics['Ast'] / sabermetrics['TO']
RegularSeasonInput['OppATORatio'] = sabermetrics['OppAst'] / sabermetrics['OppTO']

# Turnover Rate
RegularSeasonInput['TORate'] = sabermetrics['TO'] / RegularSeasonInput['Possessions']
RegularSeasonInput['OppTORate'] = sabermetrics['OppTO'] /  RegularSeasonInput['OppPossessions']

# Percentage of Shots Beyond the Arc(三分線外出手比率)
RegularSeasonInput['BArcPct'] = sabermetrics['FGA3'] / sabermetrics['FGA']
RegularSeasonInput['OppBArcPct'] = sabermetrics['OppFGA3'] /  sabermetrics['OppFGA']

# Free Throw Rate
RegularSeasonInput['FTRate'] = sabermetrics['FTA'] / sabermetrics['FGA']
RegularSeasonInput['OppFTRate'] = sabermetrics['OppFTA'] /  sabermetrics['OppFGA']

display(RegularSeasonInput)

In [41]:
seed_dict  = df_seeds.set_index(['Season', 'TeamID'])

TourneyInput = pd.DataFrame()

WinDIs = df_TourneyCompact['WTeamID']
LoseDIs = df_TourneyCompact['LTeamID']
Season = df_TourneyCompact['Season']

Winners = pd.DataFrame()
Winners[['Season', 'Team1', 'Team2']] = df_TourneyCompact[['Season', 'WTeamID', 'LTeamID']]
Winners['Result'] = 1

Losers = pd.DataFrame()
Losers[['Season', 'Team1', 'Team2']] = df_TourneyCompact[['Season', 'LTeamID', 'WTeamID']]
Losers['Result'] = 0

TourneyInput = pd.concat([Winners, Losers])
TourneyInput = TourneyInput[TourneyInput['Season'] >= 2003].reset_index(drop = True)

Team1 = []
Team2 = []

for x in range(len(TourneyInput)):
    idx = (TourneyInput['Season'][x], TourneyInput['Team1'][x])
    seed = seed_dict.loc[idx].values[0]
    if len(seed) == 4:
        seed = int(seed[1:-1])
    else:
        seed = int(seed[1:])
    Team1.append(seed)
    
    idx = (TourneyInput['Season'][x], TourneyInput['Team2'][x])
    seed = seed_dict.loc[idx].values[0]
    if len(seed) == 4:
        seed = int(seed[1:-1])
    else:
        seed = int(seed[1:])
    Team2.append(seed)

TourneyInput['Team1Seed'] = Team1
TourneyInput['Team2Seed'] = Team2
    
    
display(TourneyInput)

## Combine RegularSeasonInput & TourneyInput

In [42]:
outscores = []

for x in range(len(TourneyInput)):
    idx = (TourneyInput['Season'][x], TourneyInput['Team1'][x])
    team1score = RegularSeasonInput.loc[idx] 
    team1score['Seed'] = TourneyInput['Team1Seed'][x]
    
    idx = (TourneyInput['Season'][x], TourneyInput['Team2'][x])
    team2score = RegularSeasonInput.loc[idx] 
    team2score['Seed'] = TourneyInput['Team2Seed'][x]
    
    outscore = team1score - team2score 
    outscore['Result'] = TourneyInput['Result'][x]
    outscores.append(outscore)

outscores = pd.DataFrame(outscores)
display(outscores)

# Data Exploration

## Feature correlation with heatmap 

In [43]:
cors = round(outscores.corr(), 2)
fig3 = px.imshow(cors, height = 800, width = 800, text_auto=True)
fig3.show()

# Create Train and Test Sets

In [44]:
# Sample of submission
sample = pd.read_csv(DATA_PATH + "MSampleSubmissionStage1.csv")
sample

In [54]:
# Split out x and y(Target feature) 
x = outscores[outscores.columns[:-1]].values
y = outscores['Result'].values 

np.random.seed(1)
idx = np.random.permutation(len(x))
train_idx = idx[:int(-.2*len(x))]
test_idx = idx[int(-.2*len(x)):]

x_train = x[train_idx]
x_test = x[test_idx]
y_train = y[train_idx]
y_test = y[test_idx]

mins = x_train.min(axis=0)
maxs = x_train.max(axis=0)

x_train = (x_train - mins) / (maxs - mins)
x_test = (x_test - mins) / (maxs - mins)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [67]:
LR = LogisticRegression()
LR.fit(x_train, y_train)
LR.score(x_test, y_test)

In [55]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(x_train, y_train)
random_forest.score(x_test, y_test)

In [57]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(x_train, y_train)
decision_tree.score(x_test, y_test)

In [58]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(x_train, y_train)
knn.score(x_test, y_test)

In [62]:
gaussian = GaussianNB()
gaussian.fit(x_train, y_train)
gaussian.score(x_test, y_test)

In [63]:
svc = SVC()
svc.fit(x_train, y_train)
svc.score(x_test, y_test)

In [64]:
sgd = SGDClassifier()
sgd.fit(x_train, y_train)
sgd.score(x_test, y_test)

In [65]:
perceptron = Perceptron()
perceptron.fit(x_train, y_train)
perceptron.score(x_test, y_test)

In [70]:
sgd = SGDClassifier()
sgd.fit(x_train, y_train)
sgd.score(x_test, y_test)